**PRUEBA DE CODIGO PARA EJECUTAR EN TODOS LOS EQUIPOS DE UNA TEMPORADA**

In [4]:
import os
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
from google.colab import drive
drive.mount('/content/gdrive')

# URL inicial con la tabla de equipos
data = requests.get("https://fbref.com/en/comps/12/La-Liga-Stats")
time.sleep(10)
soup = BeautifulSoup(data.text, "html.parser")

# Extraer los links de los equipos
table = soup.select('table.stats_table')[0]
links = table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]
team_urls = [f"https://fbref.com{l}" for l in links]

# Carpeta para guardar los CSVs
folder_path = "/content/gdrive/MyDrive/Colab Notebooks/Scrapping_LaLiga"
os.makedirs(folder_path, exist_ok=True)

# Recorrer cada equipo por su índice en team_urls
for i in range(20):
    team_url = team_urls[i]
    print(f"Procesando datos para: {team_url}...")

    # Obtener datos de Scores & Fixtures
    data = requests.get(team_url)
    time.sleep(10)
    matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

    # Extraer el link para Shooting
    soup = BeautifulSoup(data.text, "html.parser")
    links = soup.find_all('a')
    links = [l.get("href") for l in links]
    links = [l for l in links if l and 'all_comps/shooting/' in l]
    shooting_url = f"https://fbref.com{links[0]}"

    # Obtener datos de Shooting
    data = requests.get(shooting_url)
    time.sleep(10)
    shooting = pd.read_html(data.text, match="Shooting")[0]
    shooting.columns = shooting.columns.droplevel()

    # Combinar los datos de Scores & Fixtures y Shooting
    team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

    # Extraer el nombre del equipo
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")

    # Ruta del CSV
    csv_path = os.path.join(folder_path, f"{team_name}.csv")

    # Si el archivo existe, combinar los datos existentes
    if os.path.exists(csv_path):
        existing_data = pd.read_csv(csv_path)
        team_data = pd.concat([existing_data, team_data], ignore_index=True)
        #### Guardar el archivo actualizado
        team_data.to_csv(csv_path, index=False)
        print(f"Datos de {team_name} actualizados en '{csv_path}'")
        print(f"{i}")
    else:
        team_data.to_csv(csv_path, index=False)
        print(f"Nuevo archivo creado para {team_name} en '{csv_path}'")
        print(f"{i}")



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


IndexError: list index out of range

In [ ]:
import os
import pandas as pd

# Directorio donde están los CSV de los equipos
la_liga_folder = "/content/gdrive/MyDrive/Colab Notebooks/La liga"

# Crear una lista con los archivos CSV de la carpeta "La liga"
la_liga_files = [f for f in os.listdir(la_liga_folder) if f.endswith('.csv')]

# Iteramos  sobre los archivos en la carpeta "La liga"
for la_liga_file in la_liga_files:

    la_liga_path = os.path.join(la_liga_folder, la_liga_file)


    data = pd.read_csv(la_liga_path)


    #AÑADIR CODIGO PARA UNIFICAR NOMBRES EN LAS COLUMNAS TEAM Y OPPONENT, QUITANDO ACENTO


    data.to_csv(la_liga_path, index=False)
    print(f"Archivo {la_liga_file} actualizado y guardado.")


IndexError: list index out of range

CODIGO COMPLETO:   

PARA EJECUTAR EN TODOS LOS EQUIPOS DE VARIAS TEMPORADAS


In [19]:
import requests
from bs4 import BeautifulSoup

url = "https://fbref.com/en/comps/12/La-Liga-Stats"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept-Language': 'en-US,en;q=0.9', # Specify the language you "prefer"
    'Referer': 'https://www.google.com/' # Faking the page you "came from"
}

try:
    print("Attempting to download with more headers...")
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    print("✅ Success! Page downloaded.")

    # Continue with your Beautiful Soup parsing
    soup = BeautifulSoup(response.text, 'html.parser')
    print("Title:", soup.find('title').text)

except requests.exceptions.RequestException as e:
    print(f"❌ Error: {e}")

Attempting to download with more headers...
❌ Error: 403 Client Error: Forbidden for url: https://fbref.com/en/comps/12/La-Liga-Stats


In [10]:
import os
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
from io import StringIO
from google.colab import drive
# Verifica si Google Drive ya está montado
if os.path.exists('/content/gdrive/MyDrive'):
    print("Google Drive ya está montado.")
else:
    print("Montando Google Drive...")
    drive.mount('/content/gdrive')

years = list(range(2025, 2020, -1))
all_matches = []
standings_url = "https://fbref.com/en/comps/12/La-Liga-Stats"
for year in years:
    # URL inicial con la tabla de equipos
    data = requests.get(standings_url)
    time.sleep(10)
    soup = BeautifulSoup(StringIO(data.text), "html.parser")

    # Extraer los links de los equipos
    table = soup.select('table.stats_table')[0]
    links = table.find_all('a')
    links = [l.get("href") for l in links]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"

    # Carpeta para guardar los CSVs

    folder_path = "/content/gdrive/MyDrive/Colab Notebooks/Scrapping_LaLiga/csv_equipos"
    os.makedirs(folder_path, exist_ok=True)

    # Recorrer cada equipo por su índice en team_urls
    for i in range(20):
        team_url = team_urls[i]
        print(f"Procesando datos para: {team_url}...")

        # Obtener datos de Scores & Fixtures
        data = requests.get(team_url)
        time.sleep(10)
        matches = pd.read_html(StringIO(data.text), match="Scores & Fixtures")[0]

        # Extraer el link para Shooting
        soup = BeautifulSoup(StringIO(data.text), "html.parser")
        links = soup.find_all('a')
        links = [l.get("href") for l in links]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        shooting_url = f"https://fbref.com{links[0]}"

        # Obtener datos de Shooting
        data = requests.get(shooting_url)
        time.sleep(10)
        shooting = pd.read_html(StringIO(data.text), match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()

        # Combinar los datos de Scores & Fixtures y Shooting
        team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

        # Extraer el nombre del equipo
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")

        # Ruta del CSV
        csv_path = os.path.join(folder_path, f"{team_name}.csv")

        # Si el archivo existe, combinar los datos existentes
        if os.path.exists(csv_path):
            existing_data = pd.read_csv(csv_path)
            team_data = pd.concat([existing_data, team_data], ignore_index=True)
            #### Guardar el archivo actualizado
            team_data.to_csv(csv_path, index=False)
            print(f"Datos de {team_name} actualizados en '{csv_path}'")
            print(f"{i}")
        else:
            team_data.to_csv(csv_path, index=False)
            print(f"Nuevo archivo creado para {team_name} en '{csv_path}'")
            print(f"{i}")



Google Drive ya está montado.


IndexError: list index out of range

#Combinación de Archivos CSV en un Único Archivo Consolidado
En esta celda, se procesan todos los archivos CSV de los equipos para combinarlos en un único archivo llamado partidos.csv. Para ello, se leen los datos de cada archivo y se extrae el nombre del equipo a partir del nombre del archivo, eliminando la extensión .csv. Esta información se añade como una nueva columna llamada "Team" en cada DataFrame. Los DataFrames resultantes se almacenan en una lista y, finalmente, se combinan en un único DataFrame utilizando pd.concat. Este DataFrame consolidado se guarda como un archivo CSV llamado partidos.csv en la misma carpeta, centralizando toda la información de los equipos en un solo lugar para facilitar su análisis.

In [ ]:
import os
import pandas as pd
import unicodedata

# Función para eliminar acentos
def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

# Directorio donde están los CSV de los equipos
csv_folder = "/content/gdrive/MyDrive/Colab Notebooks/Scrapping_LaLiga/csv_equipos"

# Directorio de Scrapping_LaLiga
la_liga_folder = "/content/gdrive/MyDrive/Colab Notebooks/Scrapping_LaLiga"

# Crear una lista con los archivos CSV de la carpeta "La liga"
la_liga_files = [f for f in os.listdir(csv_folder) if f.endswith('.csv')]

# Crear una lista para almacenar los DataFrames
all_data = []

# Iterar sobre los archivos en la carpeta "La liga"
for la_liga_file in la_liga_files:
    # Ruta completa al archivo CSV
    la_liga_path = os.path.join(csv_folder, la_liga_file)

    # Leer el archivo CSV
    data = pd.read_csv(la_liga_path)

    # Aplicar a la columna 'Opponent'
    data['Opponent'] = data['Opponent'].apply(remove_accents)


    # Eliminar columnas irrelevantes
    columns_to_drop = ["Match Report", "Captain", "Referee", "Notes"]

    # Filtrar las filas donde la columna "comp" es igual a "La Liga"
    partidos_data = data[data['Comp'] == 'La Liga']

    # Añadir el DataFrame a la lista
    all_data.append(partidos_data)

# Concatenar todos los DataFrames en uno solo
combined_data = pd.concat(all_data, ignore_index=True)

# Guardar el archivo combinado como "partidos.csv"
partidos_csv_path = os.path.join(la_liga_folder, "partidos.csv")
combined_data.to_csv(partidos_csv_path, index=False)

print(f"Archivo 'partidos.csv' creado con todos los datos.")


#Prueba  con mas Columnas#

In [ ]:
import os
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
from io import StringIO
from google.colab import drive

# Verifica si Google Drive ya está montado
if os.path.exists('/content/gdrive/MyDrive'):
    print("Google Drive ya está montado.")
else:
    print("Montando Google Drive...")
    drive.mount('/content/gdrive')

import unicodedata

# Función para eliminar acentos
def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

years = list(range(2024, 2020, -1))
all_matches = []
standings_url = "https://fbref.com/en/comps/12/La-Liga-Stats"
for year in years:
    print(year)
    # URL inicial con la tabla de equipos
    data = requests.get(standings_url)
    time.sleep(10)
    soup = BeautifulSoup(StringIO(data.text), "html.parser")

    # Extraer los links de los equipos
    table = soup.select('table.stats_table')[0]
    links = table.find_all('a')
    links = [l.get("href") for l in links]
    links = [l for l in links if '/squads/' in l]

    # Primero, extraemos los equipos y su respectiva información en la tabla

    rows = table.find_all('tr')[1:]  # Ignorar la primera fila, que son los encabezados

    # Guardamos los datos de cada equipo
    team_data = []
    for row in rows:

        # Obtener todos los elementos <th> con el atributo 'data-stat="rank"'
        rank_th_elements = row.find_all('th', {'data-stat': 'rank'})

        # Extraer el valor de 'csk' de cada uno de estos <th>
        position = None  # Inicia la variable de posición
        for th in rank_th_elements:
            csk_value = th.get('csk')  # Obtiene el valor del atributo 'csk'
            position = int(csk_value) if csk_value else None  # Asegúrate de convertirlo a entero si existe

        cols = row.find_all('td')
        team = cols[0].text.strip()  # Nombre del equipo
        team = remove_accents(team)

        # Cambiar "Betis" a "Real Betis"
        if team == "Betis":
            team = "Real Betis"

        W = int(cols[2].text.strip())  # Victorias
        D = int(cols[3].text.strip())  # Empates
        L = int(cols[4].text.strip())  # Derrotas
        GF = int(cols[5].text.strip())  # Goles a favor
        GA = int(cols[6].text.strip())  # Goles en contra
        GD = int(cols[7].text.strip())  # Diferencia de goles
        Pts = int(cols[8].text.strip())  # Puntos
        Pts_per_MP = float(cols[9].text.strip())  # Puntos por partido
        xG = float(cols[10].text.strip())  # Goles esperados
        xGA = float(cols[11].text.strip())  # Goles esperados en contra
        xGD = float(cols[12].text.strip())  # Diferencia de goles esperados
        xGD_per_90 = float(cols[13].text.strip())  # Diferencia de goles esperados por 90 minutos
        last_5 = cols[14].text.strip()  # Últimos 5 partidos


        team_data.append({
            'Team': team,
            'W': W,
            'D': D,
            'L': L,
            'GF': GF,
            'GA': GA,
            'GD': GD,
            'Pts': Pts,
            'Pts/MP': Pts_per_MP,
            'xG': xG,
            'xGA': xGA,
            'xGD': xGD,
            'xGD/90': xGD_per_90,
            'Last 5': last_5,
            'position': position
        })

    # Convertimos la lista de diccionarios a un DataFrame
    standings_df = pd.DataFrame(team_data)
    print(standings_df)

    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"

    # Carpeta para guardar los CSVs
    folder_path = "/content/gdrive/MyDrive/Colab Notebooks/Scrapping_LaLiga/csv_equipos"
    os.makedirs(folder_path, exist_ok=True)

    # Recorrer cada equipo por su índice en team_urls
    for i in range(20):
        team_url = team_urls[i]
        print(f"Procesando datos para: {team_url}...")

        # Obtener datos de Scores & Fixtures
        data = requests.get(team_url)
        time.sleep(10)
        matches = pd.read_html(StringIO(data.text), match="Scores & Fixtures")[0]

        # Extraer el link para Shooting
        soup = BeautifulSoup(StringIO(data.text), "html.parser")
        links = soup.find_all('a')
        links = [l.get("href") for l in links]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        shooting_url = f"https://fbref.com{links[0]}"

        # Obtener datos de Shooting
        data = requests.get(shooting_url)
        time.sleep(10)
        shooting = pd.read_html(StringIO(data.text), match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()

        # Combinar los datos de Scores & Fixtures y Shooting
        team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

        # Filtrar las filas donde la columna "comp" es igual a "La Liga"
        team_data = team_data[team_data['Comp'] == 'La Liga']

        team_data = team_data.copy()

        # Extraer el nombre del equipo
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")

        # Agregar los datos de la tabla de posiciones en funcion del equipo (W, D, L, etc.)
        standings = standings_df[standings_df['Team'] == team_name].iloc[0]  # Obtener la fila correspondiente al equipo
        team_data['Team'] = standings['Team']
        team_data['W'] = standings['W']
        team_data['D'] = standings['D']
        team_data['L'] = standings['L']
        team_data['GF_local'] = standings['GF']
        team_data['GA_local'] = standings['GA']
        team_data['GD_local'] = standings['GD']
        team_data['Pts'] = standings['Pts']
        team_data['Pts/MP'] = standings['Pts/MP']
        team_data['xG'] = standings['xG']
        team_data['xGA'] = standings['xGA']
        team_data['xGD'] = standings['xGD']
        team_data['xGD/90'] = standings['xGD/90']
        team_data['Last 5'] = standings['Last 5']
        team_data['position'] = standings['position']

        # Ahora, procesar la columna 'Opponent' y agregar la información del oponente
        team_data['gf_opponent'] = None
        team_data['ga_opponent'] = None
        team_data['gd_opponent'] = None

        team_data = team_data.copy()


        for idx, row in team_data.iterrows():
            opponent_name = row['Opponent']  # Extraemos el nombre del oponente
            opponent_name = remove_accents(opponent_name)  # Eliminamos acentos

            if opponent_name == "Betis":
              opponent_name = "Real Betis"


            opponent_standings = standings_df[standings_df['Team'] == opponent_name].iloc[0]  # Buscar datos del oponente
            team_data.at[idx, 'gf_opponent'] = opponent_standings['GF']
            team_data.at[idx, 'ga_opponent'] = opponent_standings['GA']
            team_data.at[idx, 'gd_opponent'] = opponent_standings['GD']
            team_data.at[idx, 'position_opponent'] = opponent_standings['position']


        # Ruta del CSV
        csv_path = os.path.join(folder_path, f"{team_name}.csv")

        # Si el archivo existe, combinar los datos existentes
        if os.path.exists(csv_path):
            existing_data = pd.read_csv(csv_path)
            team_data = pd.concat([existing_data, team_data], ignore_index=True)
            #### Guardar el archivo actualizado
            team_data.to_csv(csv_path, index=False)
            print(f"Datos de {team_name} actualizados en '{csv_path}'")
            print(f"{i}")
        else:
            team_data.to_csv(csv_path, index=False)
            print(f"Nuevo archivo creado para {team_name} en '{csv_path}'")
            print(f"{i}")



Google Drive ya está montado.
2024
               Team   W  D   L  GF  GA  GD  Pts  Pts/MP    xG   xGA   xGD  \
0       Real Madrid  15  4   2  50  20  30   49    2.33  43.0  20.1  22.9   
1   Atletico Madrid  13  6   2  35  14  21   45    2.14  36.2  18.1  18.1   
2         Barcelona  13  3   5  59  24  35   42    2.00  54.2  26.3  28.0   
3     Athletic Club  11  7   3  31  18  13   40    1.90  29.6  19.7   9.8   
4        Villarreal   9  7   5  39  32   7   34    1.62  37.1  24.5  12.6   
5          Mallorca   9  3   9  19  26  -7   30    1.43  22.8  23.5  -0.7   
6    Rayo Vallecano   7  8   6  25  24   1   29    1.38  23.9  25.5  -1.6   
7            Girona   8  4   9  29  29   0   28    1.33  26.2  24.5   1.7   
8     Real Sociedad   8  4   9  17  17   0   28    1.33  24.1  22.5   1.7   
9        Real Betis   7  7   7  23  26  -3   28    1.33  29.6  27.4   2.3   
10          Osasuna   6  9   6  25  30  -5   27    1.29  21.9  28.2  -6.3   
11          Sevilla   7  6   8  24  30  -

#Combinación de Archivos CSV en un Único Archivo Consolidado#
En esta celda, se procesan todos los archivos CSV de los equipos para combinarlos en un único archivo llamado partidos.csv. Para ello, se leen los datos de cada archivo y se extrae el nombre del equipo a partir del nombre del archivo, eliminando la extensión .csv. Esta información se añade como una nueva columna llamada "Team" en cada DataFrame. Los DataFrames resultantes se almacenan en una lista y, finalmente, se combinan en un único DataFrame utilizando pd.concat. Este DataFrame consolidado se guarda como un archivo CSV llamado partidos.csv en la misma carpeta, centralizando toda la información de los equipos en un solo lugar para facilitar su análisis.

In [ ]:
team_data.dtypes

,0
Date,object
Time,object
Comp,object
Round,object
Day,object
Venue,object
Result,object
GF,float64
GA,float64
Opponent,object


In [ ]:
team_data

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Pts,Pts/MP,xGD,xGD/90,Last 5,position,gf_opponent,ga_opponent,gd_opponent,position_opponent
0,2024-08-19,19:00,La Liga,Matchweek 1,Mon,Home,W,1.0,0.0,Espanyol,...,15,0.71,-16.8,-0.8,W L W L L,20,20,33,-13,18.0
1,2024-08-25,17:00,La Liga,Matchweek 2,Sun,Away,L,0.0,3.0,Real Madrid,...,15,0.71,-16.8,-0.8,W L W L L,20,50,20,30,1.0
2,2024-08-28,19:00,La Liga,Matchweek 3,Wed,Home,D,0.0,0.0,Leganés,...,15,0.71,-16.8,-0.8,W L W L L,20,19,29,-10,16.0
3,2024-08-31,17:00,La Liga,Matchweek 4,Sat,Away,L,0.0,7.0,Barcelona,...,15,0.71,-16.8,-0.8,W L W L L,20,59,24,35,3.0
4,2024-09-15,14:00,La Liga,Matchweek 5,Sun,Away,L,1.0,3.0,Celta Vigo,...,15,0.71,-16.8,-0.8,W L W L L,20,30,33,-3,13.0
5,2024-09-21,14:00,La Liga,Matchweek 6,Sat,Home,D,0.0,0.0,Real Sociedad,...,15,0.71,-16.8,-0.8,W L W L L,20,17,17,0,9.0
6,2024-09-24,19:00,La Liga,Matchweek 7,Tue,Away,L,1.0,2.0,Sevilla,...,15,0.71,-16.8,-0.8,W L W L L,20,24,30,-6,12.0
7,2024-09-27,21:00,La Liga,Matchweek 8,Fri,Home,L,1.0,2.0,Mallorca,...,15,0.71,-16.8,-0.8,W L W L L,20,19,26,-7,6.0
8,2024-10-05,18:30,La Liga,Matchweek 9,Sat,Home,L,1.0,2.0,Rayo Vallecano,...,15,0.71,-16.8,-0.8,W L W L L,20,25,24,1,7.0
9,2024-10-18,21:00,La Liga,Matchweek 10,Fri,Away,W,3.0,2.0,Alavés,...,15,0.71,-16.8,-0.8,W L W L L,20,25,33,-8,17.0


In [ ]:
import os
import pandas as pd
import unicodedata

# Función para eliminar acentos
def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

# Directorio donde están los CSV de los equipos
csv_folder = "/content/gdrive/MyDrive/Colab Notebooks/Scrapping_LaLiga/csv_equipos"

# Directorio de Scrapping_LaLiga
la_liga_folder = "/content/gdrive/MyDrive/Colab Notebooks/Scrapping_LaLiga"

# Crear una lista con los archivos CSV de la carpeta "La liga"
la_liga_files = [f for f in os.listdir(csv_folder) if f.endswith('.csv')]

# Crear una lista para almacenar los DataFrames
all_data = []

# Iterar sobre los archivos en la carpeta "La liga"
for la_liga_file in la_liga_files:
    # Ruta completa al archivo CSV
    la_liga_path = os.path.join(csv_folder, la_liga_file)

    # Leer el archivo CSV
    data = pd.read_csv(la_liga_path)

    # Aplicar a la columna 'Opponent'
    data['Opponent'] = data['Opponent'].apply(remove_accents)

    # Eliminar columnas irrelevantes
    columns_to_drop = ["Match Report", "Captain", "Referee", "Notes", "Formation", "Opp Formation", "Round", "Comp", "Time", "Day", "Attendance"]
    data_cleaned = data.drop(columns=columns_to_drop, errors='ignore')

    # Añadir el DataFrame a la lista
    all_data.append(data_cleaned)



# Concatenar todos los DataFrames en uno solo
combined_data = pd.concat(all_data, ignore_index=True)

# Guardar el archivo combinado como "partidos.csv"
partidos_csv_path = os.path.join(la_liga_folder, "partidos.csv")
combined_data.to_csv(partidos_csv_path, index=False)

print(f"Archivo 'partidos.csv' creado con todos los datos.")


Archivo 'partidos.csv' creado con todos los datos.


In [ ]:
combined_data.shape


(2700, 29)